In [1]:
%reload_ext autoreload

In [5]:
%autoreload
from timegan_cnn_DGER import *
from utils import *
from trainers import *


trainset = TimeGANDatasetSinus(num=600, seq_len=100, features=3, temporal=True)
testset = TimeGANDatasetSinus(num=300, seq_len=100, features=3, temporal=True)
print(len(trainset))
print(trainset[0][0].size())


600
torch.Size([100, 3])


In [ ]:
%autoreload
import neptune.new as neptune

run = neptune.init_run(
    project="kohmann/timeGAN",
    name="timegan_cnn_DGER",
    tags=["cnn","DGER" "batchnorm", "temporal"],
    description="DGER temporal test",
    source_files=["timegan_cnn_DGER.py"],
    capture_hardware_metrics=False,
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI3YjFjNGY5MS1kOWU1LTRmZjgtOTNiYS0yOGI2NDdjZGYzNWUifQ==",
)


device = "cuda" if torch.cuda.is_available() else "cpu"

params = {
    "model_name": "model_cnn_dger.pt",
    "n_epochs": 800,
    "l_rate": 0.0001,
    "batch_size": 64,
    "num_layers": 2,
    "hidden_dim": 20,
    "Z_dim": 100,
    "dis_thresh": 0.15,
    "dataset": "sinus",
    "feature_dim": trainset[0][0].size(1),
    "max_seq_len": trainset[0][0].size(0),
    "trainset_size": len(trainset),
    "device": device,
    "optimizer": "Adam",
}
run["parameters"] = params




model = TimeGAN(feature_dim=params["feature_dim"],
                hidden_dim=params["hidden_dim"],
                num_layers=params["num_layers"],
                padding_value=0.,
                Z_dim=params["Z_dim"],
                max_seq_len=params["max_seq_len"],
                batch_size=params["batch_size"],
                device=torch.device(params["device"]))

timegan_trainer(model,
                trainset,
                batch_size=params["batch_size"],
                device=torch.device(params["device"]),
                learning_rate=params["l_rate"],
                n_epochs=params["n_epochs"],
                max_seq_len=params["max_seq_len"],
                dis_thresh=params["dis_thresh"],
                neptune_logger=run,
                model_name=params["model_name"]
                )
# Generate random synthetic data
gen_z = timegan_generator(model, torch.tensor(testset.T), params["model_name"], torch.device(device), testset[0][0].size(0), 100)


r = np.array([data[0].numpy() for data in testset])
f_pca = visualization(r, gen_z, 'pca')
run["PCA"].upload(f_pca)
plt.close(f_pca)
f_tsne = visualization(r, gen_z, 'tsne')
run["tsne"].upload(f_tsne)
plt.close(f_tsne)

run["mode_collapse"] = modeCollapseEvaluator(r, gen_z)
run["model_checkpoint"].upload(params["model_name"])

run.stop()

https://app.neptune.ai/kohmann/timeGAN/e/TIMEGAN-99
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.

Start Embedding Network Training


Epoch: 799, Loss: 0.0499: 100%|██████████| 800/800 [00:56<00:00, 14.13it/s]



Start Training with Supervised Loss Only


Epoch: 544, Loss: 0.0718:  68%|██████▊   | 544/800 [00:45<00:20, 12.45it/s]

In [ ]:
gen_z = timegan_generator(model, torch.tensor(testset.T), "model_cnn_dg_meta.pt", torch.device(device), testset[0][0].size(0),
                          100)

In [ ]:
sample = 12
plt.plot(gen_z[sample][:, 0], label="gen_sin1")
plt.plot(gen_z[sample][:, 1], label="gen_sin2")
plt.plot(gen_z[sample][:, 2], label="gen_sin3")
plt.title("Generated sampels")
plt.legend()

In [ ]:
sample = 1
real_combo = create_sin3(gen_z[sample][:,0], gen_z[sample][:,1], temporal=True)
real_comb_scaled = minmaxscaler().fit_transform(torch.tensor([real_combo]))[0]
gen_combo = gen_z[sample][:,2]
plt.plot(real_comb_scaled, label="true sim3")
plt.plot(gen_combo, label="generated sim3")
plt.title("sim3 - true vs generated")
plt.legend()

In [ ]:
sample = trainset.__getitem__(5)
plt.plot(sample[0][:,0], label="sin1")
plt.plot(sample[0][:,1], label="sin2")
plt.plot(sample[0][:,2], label="(sin1 + sin2)/2")
plt.title("Real data")
plt.legend()

In [ ]:
real_comb = create_sin3(gen_z[sample][:,0], gen_z[sample][:,0], temporal=True)
real_comb_scaled = minmaxscaler().fit_transform(torch.tensor([real_combo]))
plt.plot(real_comb_scaled[0])

### CNN layer testing

In [4]:
def printLayerShapes(layers, input):
    for layer in layers:
        output = layer(input)
        print(f"{type(layer).__name__}: {list(input.size())} --> {list(output.size())}")
        input = output

In [5]:
import torch
import torch.nn as nn
input = torch.randn(20, 100, 20)


## Discrininator
layers = nn.Sequential(
            nn.Conv1d(in_channels=100, out_channels=20, kernel_size=(5), stride=2, bias=False)
            #,nn.Flatten(start_dim=1)
            ,nn.BatchNorm1d(20)
            ,nn.LeakyReLU()
            ,nn.Conv1d(in_channels=20, out_channels=40, kernel_size=(7), stride=2, bias=False)
            #,nn.Flatten(start_dim=1)
            ,nn.BatchNorm1d(40)
            ,nn.LeakyReLU()
            ,nn.Flatten(start_dim=1)
            ,nn.Linear(40, 1)
        )
printLayerShapes(layers, input)

Conv1d: [20, 100, 20] --> [20, 20, 8]
BatchNorm1d: [20, 20, 8] --> [20, 20, 8]
LeakyReLU: [20, 20, 8] --> [20, 20, 8]
Conv1d: [20, 20, 8] --> [20, 40, 1]
BatchNorm1d: [20, 40, 1] --> [20, 40, 1]
LeakyReLU: [20, 40, 1] --> [20, 40, 1]
Flatten: [20, 40, 1] --> [20, 40]
Linear: [20, 40] --> [20, 1]


In [6]:
## Generator
input = torch.randn(20, 100, 100)

layers = nn.Sequential(
            nn.Conv1d(in_channels=100, out_channels=100, kernel_size=(6), stride=2, bias=False)
            , nn.BatchNorm1d(100)
            , nn.LeakyReLU()
            , nn.Conv1d(in_channels=100, out_channels=100, kernel_size=(9), stride=2, bias=True)
        )

printLayerShapes(layers, input)

Conv1d: [20, 100, 100] --> [20, 100, 48]
BatchNorm1d: [20, 100, 48] --> [20, 100, 48]
LeakyReLU: [20, 100, 48] --> [20, 100, 48]
Conv1d: [20, 100, 48] --> [20, 100, 20]


In [7]:
# Recovery

## Discrininator
layers = nn.Sequential(
            nn.Conv1d(in_channels=100, out_channels=100, kernel_size=(5), stride=2, bias=False)
            #,nn.Flatten(start_dim=1)
            ,nn.BatchNorm1d(100)
            ,nn.LeakyReLU()
            ,nn.Conv1d(in_channels=100, out_channels=100, kernel_size=(3), stride=2, bias=True)

        )
printLayerShapes(layers, input)


Conv1d: [20, 100, 100] --> [20, 100, 48]
BatchNorm1d: [20, 100, 48] --> [20, 100, 48]
LeakyReLU: [20, 100, 48] --> [20, 100, 48]
Conv1d: [20, 100, 48] --> [20, 100, 23]


In [8]:
output = layers[0](input)

In [9]:
## Embedder
input = torch.randn(20, 100, 3)

layers = nn.Sequential(
            #nn.Conv1d(in_channels=100, out_channels=100, kernel_size=(3), stride=2, bias=False)
            nn.ConvTranspose1d(in_channels=100, out_channels=100, kernel_size=(6), stride=2, bias=False)
            , nn.BatchNorm1d(100)
            , nn.LeakyReLU()
            , nn.ConvTranspose1d(in_channels=100, out_channels=100, kernel_size=(2), stride=2, bias=False)
        )

printLayerShapes(layers, input)

ConvTranspose1d: [20, 100, 3] --> [20, 100, 10]
BatchNorm1d: [20, 100, 10] --> [20, 100, 10]
LeakyReLU: [20, 100, 10] --> [20, 100, 10]
ConvTranspose1d: [20, 100, 10] --> [20, 100, 20]


In [10]:
## Recovery
input = torch.randn(20, 100, 20)
layers = nn.Sequential(
            nn.Conv1d(in_channels=100, out_channels=100, kernel_size=5, stride=2, bias=False)
            , nn.BatchNorm1d(100)
            , nn.LeakyReLU()
            , nn.Conv1d(in_channels=100, out_channels=100, kernel_size=3, stride=2, bias=True)
            #, nn.Flatten(start_)
        )
printLayerShapes(layers, input)

Conv1d: [20, 100, 20] --> [20, 100, 8]
BatchNorm1d: [20, 100, 8] --> [20, 100, 8]
LeakyReLU: [20, 100, 8] --> [20, 100, 8]
Conv1d: [20, 100, 8] --> [20, 100, 3]
